In [ ]:
# import the necessary packages
import os
import pandas as pd
import numpy as np 
import glob
import matplotlib.pyplot as plt 
import geopandas as gpd
from shapely.geometry import LineString


In [ ]:
# Specify the path to the folder which has the survey data
path = r"path to your survey data"

# Normally there are three files: QP : Cross section (Querprofile) , LS: river banks (Langstructure) and EP: Single points (Einzelpunket)
LS_gdf = gpd.read_file(os.path.join(path, 'LS.shp'))
QP_gdf = gpd.read_file(os.path.join(path, 'QP.shp'))
EP_gdf = gpd.read_file(os.path.join(path, 'EP.shp'))


In [ ]:
# Specify the path to your river channel shapefiel (Gewässernetz.shp)
Gewaesser_netz_path=r'Path to your river channel layer'

# Load the river channel shapefile into a GeoDataFrame
Gewaesser_netz_gdf=gpd.read_file(Gewaesser_netz_path)

# Plot the river channel GeoDataFrame
Gewaesser_netz_gdf.plot();

In [ ]:
# Check if the layers have the same projection and print the EPSG codes

# Print the EPSG code of the projection for the river channel GeoDataFrame
print('Gewässernetz Proj:', Gewaesser_netz_gdf.crs.to_epsg())

# Print the EPSG code of the projection for the cross section GeoDataFrame
print('QP Proj:', QP_gdf.crs.to_epsg())

# Print the EPSG code of the projection for the river banks GeoDataFrame
print('LS Proj:', LS_gdf.crs.to_epsg())

# Print the EPSG code of the projection for the single points GeoDataFrame
print('EP Proj:', EP_gdf.crs.to_epsg())


In [ ]:
# Display the first few rows of the river channel GeoDataFrame
Gewaesser_netz_gdf.head()

In [ ]:
# Display the data types of columns in the river channel GeoDataFrame
Gewaesser_netz_gdf.dtypes

In [ ]:
# Display the first few rows of the cross section GeoDataFrame
QP_gdf.head()

In [ ]:
# Display the data types of columns in the cross section GeoDataFrame
QP_gdf.dtypes

In [ ]:
# Convert the 'GKZ' (River ID: Gewässerkennzahl) column in the cross section GeoDataFrame to float type
QP_gdf['GKZ'] = QP_gdf['GKZ'].astype(float)

In [ ]:
# Initialize GeoDataFrames to save the results
# Create an empty GeoDataFrame with columns and CRS matching QP_gdf
no_intersection = gpd.GeoDataFrame(columns=QP_gdf.columns, crs=QP_gdf.crs)

# Create another empty GeoDataFrame with columns and CRS matching QP_gdf
different_values = gpd.GeoDataFrame(columns=QP_gdf.columns, crs=QP_gdf.crs)

# Iterate over each row (cross section) in QP_gdf to check for intersections and attribute matches
for idx, QP in QP_gdf.iterrows():
    # Find all river channels (GEWAESSER) that intersect with the current cross section (QP['geometry'])
    matching_QP = Gewaesser_netz_gdf[Gewaesser_netz_gdf.intersects(QP['geometry'])]
    
    # If no intersections are found, add the current cross section (QP) to no_intersection GeoDataFrame
    if matching_QP.empty:
        no_intersection = pd.concat([no_intersection, QP.to_frame().T], ignore_index=True)
    else:
        # Check if there are matching 'GEWAESSER' attributes within the intersected geometries
        if not any(matching_QP['GEWAESSER'] == QP['GKZ']):
            different_values = pd.concat([different_values, QP.to_frame().T], ignore_index=True)


In [ ]:
# display the result
no_intersection

In [ ]:
# save the results as a shapefile
no_intersection,to_file('No_intersection.shp')

In [ ]:
# display the result
different_values

In [ ]:
# save the results as a shapefile
different_values.to_file('Crosssection_with_different_River_ID.shp')